In [ ]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [ ]:
data0 = pd.read_csv("../input/star-type-classification/Stars.csv")
data0[0:2]

In [ ]:
data0.columns

In [ ]:
data0['Type'].unique()

In [ ]:
data0['Color'].unique()

In [ ]:
data0['Spectral_Class'].unique()

In [ ]:
list_cols = ['Spectral_Class','Color']
ce_ohe = ce.OrdinalEncoder(cols=list_cols,handle_unknown='impute')
data1 = ce_ohe.fit_transform(data0)
data1[0:2]

In [ ]:
n=len(data1)
print(n)

In [ ]:
N=[]
for i in range(n):
    N+=[i]
random.shuffle(N)

In [ ]:
datay=data1['Type']
data2=data1.drop(['Type'],axis=1)
data2[0:2]

In [ ]:
data3=np.array(data2)
datay3=np.array(datay)

In [ ]:
train=data3[N[0:(n//10)*8]]
test=data3[N[(n//10)*8:]]
trainy=datay3[N[0:(n//10)*8]]
testy=datay3[N[(n//10)*8:]]

In [ ]:
df_columns=['Temperature','L','R','A_M','Color','Spectral_Class']

In [ ]:
train_df=pd.DataFrame(train)
test_df=pd.DataFrame(test)

In [ ]:
train_df.columns=df_columns
test_df.columns=df_columns

In [ ]:
def create_numeric_feature(input_df):
    use_columns = df_columns
    return input_df[use_columns].copy()

In [ ]:
create_numeric_feature(train_df)[0:2]

In [ ]:
from contextlib import contextmanager
from time import time

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' '):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [ ]:
from tqdm import tqdm

def to_feature(input_df):

    processors = [
        create_numeric_feature,
    ]
    
    out_df = pd.DataFrame()
    
    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='create' + func.__name__ + ' '):
            _df = func(input_df)

        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
        
    return out_df

In [ ]:
train_feat_df = to_feature(train_df)
test_feat_df = to_feature(test_df)

In [ ]:
train_feat_df[0:2]

In [ ]:
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error

def fit_lgbm(X, 
             y, 
             cv, 
             params: dict=None, 
             verbose: int=50):

    if params is None:
        params = {}

    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMRegressor(**params)
        
        with Timer(prefix='fit fold={} '.format(i)):
            clf.fit(x_train, y_train, 
                    eval_set=[(x_valid, y_valid)],  
                    early_stopping_rounds=100,
                    verbose=verbose)

        pred_i = clf.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        print(f'Fold {i} RMSLE: {mean_squared_error(y_valid, pred_i) ** .5:.4f}')
        print()

    score = mean_squared_error(y, oof_pred) ** .5
    print('-' * 50)
    print('FINISHED | Whole RMSLE: {:.4f}'.format(score))
    return oof_pred, models

In [ ]:
params = {
    'objective': 'rmse', 
    'learning_rate': .1,
    'reg_lambda': 1.,
    'reg_alpha': .1,
    'max_depth': 5, 
    'n_estimators': 10000, 
    'colsample_bytree': .5, 
    'min_child_samples': 10,
    'subsample_freq': 3,
    'subsample': .9,
    'importance_type': 'gain', 
    'random_state': 71,
}

In [ ]:
y = trainy

In [ ]:
from sklearn.model_selection import KFold

fold = KFold(n_splits=5, shuffle=True, random_state=71)
cv = list(fold.split(train_feat_df, y))
oof, models = fit_lgbm(train_feat_df.values, y, cv, params=params, verbose=500)

In [ ]:
fig,ax = plt.subplots(figsize=(8,8))
ax.set_xlabel('oof',fontsize=20)
ax.set_ylabel('train_y',fontsize=20)
ax.scatter(oof,y)

In [ ]:
def visualize_importance(models, feat_train_df):

    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importances_
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], 
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x='feature_importance', 
                  y='column', 
                  order=order, 
                  ax=ax, 
                  palette='viridis', 
                  orient='h')
    ax.tick_params(axis='x', rotation=0)
    ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()
    return fig, ax

fig, ax = visualize_importance(models, train_feat_df)

In [ ]:
def revert_to_real(preda):
    predb = pd.Series(preda).apply(lambda x:round(x))
    return predb

In [ ]:
pred0 = np.array([model.predict(test_feat_df.values) for model in models])
pred1 = np.mean(pred0, axis=0)
pred2 = revert_to_real(pred1)
print(pred1[0:5])
print(np.array(pred2)[0:5])

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.histplot(pred1, label='Test Predict', ax=ax, color='black')
sns.histplot(oof, label='Train Predict', ax=ax, color='C1')
ax.legend()
ax.grid()